# Example Notebook

## Example 1: Building YAML config from a Python dictionary
First, we need to import `uwtools.api.config` from the uwtools python package.

In [1]:
from uwtools.api import config

The `config.get_yaml_config` method can create a `YAMLconfig` object when given a Python dictionary.

In [2]:
# An example Python dictionary
dictionary = {"values":{"date":20240105, "greeting":"Good Night", "recipient":"Moon", "repeat":2}}

# Build a YAMLconfig object from the dictionary
config_yaml = config.get_yaml_config(dictionary)
print(config_yaml)

values:
  date: 20240105
  greeting: Good Night
  recipient: Moon
  repeat: 2


## Example 2: Rendering a template with uwtools
Next, let's look at using the `template` tool to render a Jinja2 template.

In [3]:
from uwtools.api import template

We have a Jinja2 template file in `fixtures/user_config.yaml` that looks like this:

In [4]:
%%bash
cat ./fixtures/user_config.yaml

user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}


We can use another yaml file that contains the values we want to add to the template to complete it:

In [5]:
%%bash
cat ./fixtures/user_values.yaml

first: John
last: Doe
food: burritos


Using `template.render` we can render the `user_config.yaml` file using the values supplied by the `user_values.yaml` to create a complete and ready to use config file.

In [6]:
# path of Jinja2 template to update
source = "./fixtures/user_config.yaml"

# values to add
vals = "./fixtures/user_values.yaml"

# destination of the rendered file
target = "./fixtures/rendered_config.yaml"

# render the template 
print(template.render(values_src=vals, values_format="yaml", input_file=source, output_file=target))

user:
  name: John Doe
  favorite_food: burritos


Let's take a look at the rendered file:

In [7]:
%%bash
cat ./fixtures/rendered_config.yaml

user:
  name: John Doe
  favorite_food: burritos


## Example 3: Comparing two config files
Let's explore using the `config.compare()` method to compare two config files.

We again need to start by importing the `uwtools.api.config` from the `uwtools` python package.

In [8]:
from uwtools.api import config
from uwtools.api.logging import use_uwtools_logger
use_uwtools_logger()

Please review the [config.compare() documentation](https://uwtools.readthedocs.io/en/main/sections/user_guide/api/config.html#uwtools.api.config.compare) for full information on this function's arguments.

For example, let's compare two Fortran namelist files with differences:

In [9]:
%%bash
cat ./fixtures/config_test_file_a.nml

&foo n=88, s="string" /

In [10]:
%%bash
cat ./fixtures/config_test_file_b.nml

&FOO
  S = "string"
  N = 99
/

In [11]:
file_a = "./fixtures/config_test_file_a.nml"
different_file_b = "./fixtures/config_test_file_b.nml"
config.compare(file_a, different_file_b)

[2024-07-18T19:27:40]     INFO - fixtures/config_test_file_a.nml
[2024-07-18T19:27:40]     INFO + fixtures/config_test_file_b.nml
[2024-07-18T19:27:40]     INFO ---------------------------------------------------------------------
[2024-07-18T19:27:40]     INFO foo:               n:  - 88 + 99


False

The `config()` method returns `False` to denote the files are different. The UW logger shows the difference, of one file containing `n = 88`, and one file containing `n = 99`.

Now to compare two semantically equivalent files:

In [12]:
%%bash
cat ./fixtures/config_test_file_a.nml

&foo n=88, s="string" /

In [13]:
%%bash
cat ./fixtures/config_test_file_c.nml

&FOO
  S = "string"
  N = 88
/

In [14]:
file_a = "./fixtures/config_test_file_a.nml"
identical_file_c = "./fixtures/config_test_file_c.nml"
config.compare(file_a, identical_file_c)

[2024-07-18T19:27:40]     INFO - fixtures/config_test_file_a.nml
[2024-07-18T19:27:40]     INFO + fixtures/config_test_file_c.nml
[2024-07-18T19:27:40]     INFO ---------------------------------------------------------------------


True

The `config()` method returns `True` to denote the files are semantically equivalent.